## Resource-Constrained Project Scheduling Problem with Transfer Times

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)). [Problem defintion from Vilem Heinz](https://www.overleaf.com/1895916189bqhxjfkpwvws#151a3f)

### Problem Definiton

The Resource-Constrained Project Scheduling Problem with Transfer Times (RCPSPTT) is an extension of the RCPSP problem. In RCPSP, activities are executed using available resources, precedence relationships between them must be adhered to, and the total makespan is minimized. Compared to the RCPSP, RCPSPTT introduces transfer times between activities. This means that resources do not have to be directly available after the activity that uses them finishes; rather, a transfer of resources to the following activity might be required. The transfer time depends on both the activities and the resource transferred.

Formally, the RCPSPTT problem is defined as follows:

Let $\mathcal{A}=\{0,1,\ldots,\hat{A}\}$, $\ \hat{A}\in\mathbb{N}$, be a set of activities with indices $i,j\in\mathcal{A}$.

Let $P\in\mathbb{N}^{|\mathcal{A}|}$ be a vector of activity processing times.

Let $\mathcal{E}=\{(i,j)\mid \text{activity } i \text{ precedes activity } j\}$ be a set of precedences.

Let $\mathcal{R}=\{0,1,\ldots,\hat{R}\}$, $\ \hat{R}\in\mathbb{N}$, be a set of primary resources indexed by $r\in\mathcal{R}$.

Let $C\in\mathbb{N}^{|\mathcal{R}|}$ be a vector of primary resource capacities.

Let $\mathbf{Q}\in\mathbb{Z}_{{\ge 0}}^{|\mathcal{A}|\times|\mathcal{R}|}$ be the activity–resource consumption matrix, with entries $\mathbf{Q}_{i,r}$ denoting the consumption of resource $r$ by activity $i$ ($i=0 \Rightarrow \mathbf{Q}_{i,r}=C_r$).

Let $\Delta\in\mathbb{Z}_{\ge 0}^{|\mathcal{A}|\times|\mathcal{A}|\times|\mathcal{R}|}$ be a matrix of transfer delays, with entries $\Delta_{i,j,r}$ denoting the delay required if resource $r$ is transferred between activities $i$ and $j$.

Let $\mathcal{T} \subseteq \mathcal{A} \times \mathcal{A} \times \mathcal{R}$ be the set of feasible resource transfers, where $(i,j,r) \in \mathcal{T}$ if:
* $i \neq j$ (no self-transfers)
* $(j,i) \notin \mathcal{E}$ (no reverse precedence relationship)
* Either $\mathbf{Q}_{i,r} > 0$ or $i = 0$ (source activity has resource $r$)
* Either $\mathbf{Q}_{j,r} > 0$ or $j = |\mathcal{A}|-1$ (target activity needs resource $r$ or is sink)

### CP Optimizer Formulation

$$
\begin{aligned}
\min \quad
& \operatorname{end}(a_{|\mathcal{A}|-1})
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \operatorname{endBeforeStart}(a_i, a_j),
\qquad & \forall (i, j) \in \mathcal{E}
\quad & \text{(2)} \\[1mm]
& \sum_{j:(0, j, r) \in \mathcal{T}} f_{0, j, r} = C_r,
\qquad & \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(3)} \\[1mm]
& f_{i,j,r} \ge 1 \implies \operatorname{presenceOf}(z_{i,j,r}) = 1,
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r} = 0
\quad & \text{(4)} \\[2mm]
& f_{i,j,r} = \operatorname{heightAtStart}(z_{i,j,r}, \operatorname{pulse}(z_{i,j,r}, (0, U_{i,j,r}))),
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r} > 0
\quad & \text{(5)} \\[2mm]
& \sum_{\substack{j, k:(j, i, k) \in \mathcal{T}, k=r}} f_{j, i, k} = Q_{i, r},
\qquad & \forall i \in \{1, \dots, |\mathcal{A}| - 1\}, \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(6)} \\[2mm]
& \sum_{\substack{j, k:(i, j, k) \in \mathcal{T}, k=r}} f_{i, j, k} = Q_{i, r},
\qquad & \forall i \in \{0, \dots, |\mathcal{A}| - 2\}, \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(7)} \\[2mm]
& \operatorname{endBeforeStart}(a_i, z_{i, j, r}) \land \operatorname{endBeforeStart}(z_{i, j, r}, a_j),
\qquad & \forall (i, j, r) \in \mathcal{T}
\quad & \text{(8)} \\[2mm]
& \sum_{i: \mathbf{Q}_{i, r} > 0} \operatorname{pulse}(a_i, \mathbf{Q}_{i, r}) + \sum_{\substack{(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} > 0, k=r}} \operatorname{pulse}(z_{i, j, r}, (0, U_{i, j, r})) \le C_r,
\qquad & \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(9)} \\[2mm]
& a_i: \text{mandatory interval var, length } p_i,
\qquad & \forall i \in \mathcal{A}
\quad & \text{(10a)} \\[1mm]
& f_{i,j,r} \in \mathbb{Z}, f_{i,j,r} \in [0, U_{i,j,r}],
\qquad & \forall (i, j, r) \in \mathcal{T}
\quad & \text{(10b)} \\[1mm]
& z_{i,j,r}: \text{optional interval var, length } \Delta_{i,j,r},
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r} > 0
\quad & \text{(10c)}
\end{aligned}
$$

**Objective:**
  - (1) Minimize the makespan (the end time of the final sink activity $a_{|\mathcal{A}|-1}$).

**Modeling Constraints:**
  - (2) Precedence relations: Enforces precedence relations $\mathcal{E}$ between activities, ensuring an activity must finish before its successor can start.
  - (3) Source flow initialization: Ensures the total resource flow $r$ from the source node (activity 0) via all transfers ($f_{0,j,r}$) equals the total resource capacity $C_r$.
  - (4) Transfer activation for instantaneous transfers: For instantaneous transfers ($\Delta_{i,j,r} = 0$), if flow $f_{i,j,r} \geq 1$ exists, then the optional interval $z_{i,j,r}$ must be present to enable the transfer.
  - (5) Flow-height linkage for durative transfers: For durative transfers ($\Delta_{i,j,r} > 0$), the flow variable $f_{i,j,r}$ is constrained to equal the height of the pulse function at the start of the transfer interval $z_{i,j,r}$.
  - (6) Flow conservation (into activity): Ensures the total resource $r$ received by activity $i$ (summed from all incoming transfers, both durative and instantaneous) equals its required quantity $Q_{i,r}$.
  - (7) Flow conservation (out of activity): Ensures the total resource $r$ sent from activity $i$ (summed via all outgoing transfers) equals the quantity $Q_{i,r}$ that the activity processed (or $C_r$ for the source node $i=0$).
  - (8) Temporal linking: Enforces the time sequencing for transfers. A transfer $z_{i,j,r}$ must start after activity $a_i$ ends, and activity $a_j$ must start after transfer $z_{i,j,r}$ ends.
  - (9) Resource capacity (cumulative constraint): Ensures that at any point in time, the sum of resources consumed by all active activities ($\operatorname{pulse}(a_i, \mathbf{Q}_{i,r})$) and resources in transit during active durative transfers ($\operatorname{pulse}(z_{i,j,r}, (0, U_{i,j,r}))$) does not exceed the capacity $C_r$ for resource $r$.

**Variable Definitions:**
  - (10a) $a_i$: A mandatory interval variable representing the execution of activity $i$ with fixed length $p_i$.
  - (10b) $f_{i,j,r}$: An integer variable representing the amount of resource $r$ flowing from activity $i$ to activity $j$. For instantaneous transfers ($\Delta_{i,j,r}=0$), this is a decision variable. For durative transfers ($\Delta_{i,j,r}>0$), this is linked to the pulse height via constraint (5).
  - (10c) $z_{i,j,r}$: An optional interval variable representing the durative transfer of resource $r$ from activity $i$ to activity $j$, with fixed length $\Delta_{i,j,r}$. Only defined for $(i,j,r) \in \mathcal{T}$ where $\Delta_{i,j,r} > 0$.

**Helper Expressions:**
  - $\operatorname{pulse}(z_{i,j,r}, (0, U_{i,j,r}))$: Creates a step function over the interval $z_{i,j,r}$ that contributes $U_{i,j,r}$ units of resource $r$ at each time point when the transfer is active.
  - $\operatorname{heightAtStart}(z_{i,j,r}, \operatorname{pulse}(z_{i,j,r}, (0, U_{i,j,r})))$: Returns the height of the pulse function at the start of interval $z_{i,j,r}$.

**Parameters (Input Data):**
  - $\mathcal{A}$: The set of activities (including source activity 0 and sink activity $|\mathcal{A}|-1$).
  - $\mathcal{E}$: The set of precedence relations (ordered pairs of activities).
  - $\mathcal{R}$: The set of primary resources.
  - $\mathcal{T}$: The set of feasible transfers (triples of source activity, target activity, resource).
  - $p_i$: The fixed processing time (duration) of activity $a_i$.
  - $C_r$: The maximum capacity available for resource $r$.
  - $\Delta_{i,j,r}$: The duration (delay) for a transfer $(i, j, r)$. If $\Delta_{i,j,r} = 0$, the transfer is instantaneous.
  - $U_{i,j,r}$: The upper bound (maximum flow capacity) of the transfer $(i, j, r)$, computed as:
    - For source activity ($i=0$): $U_{0,j,r} = \min(C_r, \text{max\_flow\_limit})$
    - For other activities: $U_{i,j,r} = \min(Q_{i,r}, C_r, \text{max\_flow\_limit})$
  - $Q_{i,r}$: The quantity of resource $r$ required by/consumed by activity $i$ (with $Q_{0,r} = C_r$ for the source).

### DOCPLEX Implementation

#### Imports

In [10]:
import re
from pathlib import Path
import numpy as np
from docplex.cp.model import CpoModel

#### Parsing the data file

In [11]:
def parse_rcpsp_psplib(filepath):
    """
    Parses a .sm file (PSPLIB format for RCPSP with transfer times)
    and returns a dictionary with the project data.
    """
    with open(filepath, 'r') as f:
        content = f.read()

    data = {}
    match = re.search(r'jobs \(incl\. supersource/sink \):\s*(\d+)', content)
    data['n_jobs'] = int(match.group(1)) if match else 0
    match = re.search(r' - renewable\s*:\s*(\d+)', content)
    data['n_resources'] = int(match.group(1)) if match else 0
    n_jobs = data['n_jobs']
    n_res = data['n_resources']
    data['precedence_arcs'] = []
    prec_start = content.find('PRECEDENCE RELATIONS:')
    prec_end = content.find('****************', prec_start)
    prec_section = content[prec_start:prec_end]
    
    for line in prec_section.splitlines()[2:]:
        if not line.strip():
            continue
        parts = [int(p) for p in line.strip().split()]
        predecessor = parts[0]
        successors = parts[3:]
        for succ in successors:
            # Convert 1-based index from file to 0-based index
            data['precedence_arcs'].append((predecessor - 1, succ - 1))

    data['durations'] = []
    data['demands'] = []
    req_start = content.find('REQUESTS/DURATIONS:')
    req_end = content.find('****************', req_start)
    req_section = content[req_start:req_end]

    for line in req_section.splitlines()[3:]:
        if not line.strip():
            continue
        parts = [int(p) for p in line.strip().split()]
        data['durations'].append(parts[2])
        data['demands'].append(parts[3:]) # The rest are resource demands

    cap_start = content.find('RESOURCEAVAILABILITIES:')
    cap_end = content.find('****************', cap_start)
    cap_section = content[cap_start:cap_end]

    cap_line = cap_section.splitlines()[2]
    data['capacities'] = [int(p) for p in cap_line.strip().split()]
    data['transfer_times'] = []
    current_pos = cap_end

    for _ in range(n_res):
        tt_start = content.find('TRANSFERTIMES', current_pos)
        tt_end = content.find('****************', tt_start)
        tt_section = content[tt_start:tt_end]
        
        matrix = []
        lines = tt_section.splitlines()[3:]
        
        for i in range(n_jobs):
            line = lines[i]
            parts = [int(p) for p in line.strip().split()]
            matrix.append(parts[1:])
            
        data['transfer_times'].append(matrix)
        current_pos = tt_end
    return data

#### Helper functions and data structures

In [12]:
def compute_transitive_closure(edges, n_jobs):
    """
    Computes the transitive closure of the precedence graph using Floyd-Warshall algorithm. 
    Returns all precedence relationships (both direct and transitive) as (i, j) tuples.
    """
    adj = np.zeros((n_jobs, n_jobs), dtype=bool)
    for i, j in edges:
        adj[i, j] = True
    # Floyd-Warshall
    for k in range(n_jobs):
        for i in range(n_jobs):
            for j in range(n_jobs):
                adj[i, j] = adj[i, j] or (adj[i, k] and adj[k, j])
    return [(i, j) for i in range(n_jobs) for j in range(n_jobs) if adj[i, j]]

def compute_possible_transfers(abs_A, abs_R, Q, C, E, max_flow_limit=1000):
    """
    Generates the set T (Transfers) and calculates U (Upper bounds).
    """
    T = {}  # Maps (i, j, r) -> U_ijr
    for i in range(abs_A):
        for j in range(abs_A):
            # Transfers possible if not same node and not strictly reverse precedence
            if i != j and (j, i) not in set(E):
                for r in range(abs_R):
                    # Check if resource demand exists at ends or if source/sink
                    if (Q[i, r] > 0 or i == 0) and (Q[j, r] > 0 or j == abs_A - 1):
                        # Calculate U_ijr (Upper bound)
                        # For source (i=0), flow is limited by Capacity C[r]
                        # For others, limited by demand Q[i,r] or Capacity C[r]
                        max_poss = C[r] if i == 0 else min(Q[i, r], C[r])
                        # Store into Set T with associated U
                        T[(i, j, r)] = min(max_poss, max_flow_limit)
    return T

In [13]:
filename = "../data/rcpsptt/j301_a.sm"
filepath = Path(filename)

# Parse data
data = parse_rcpsp_psplib(filepath)
abs_A = data['n_jobs']
abs_R = data['n_resources']
p = data['durations']
C = np.array(data['capacities'])
Q = np.array(data['demands'])
E = compute_transitive_closure(data['precedence_arcs'], abs_A)
Delta = np.array(data['transfer_times']).transpose((1, 2, 0))
T = compute_possible_transfers(abs_A, abs_R, Q, C, E)

#### Create model and variables

In [14]:
mdl = CpoModel(name='RCPSPTT')

# (10a): a_i (mandatory interval variables)
a = [mdl.interval_var(size=p[i], name=f'a_{i}') for i in range(abs_A)]

# (10b): f_{i,j,r} (integer flow variables)
f = {(i, j, r): mdl.integer_var(min=0, max=U_ijr, name=f'f_{i}_{j}_{r}')
     for (i, j, r), U_ijr in T.items()}

# (10c): z_{i,j,r} (optional interval variables for transfers)
z = {(i, j, r): mdl.interval_var(size=int(Delta[i, j, r]), optional=True,
                                 name=f'z_{i}_{j}_{r}')
     for (i, j, r) in T.keys()}

#### Add constraints and define objective

In [15]:
# (1): Minimize makespan
mdl.add(mdl.minimize(mdl.end_of(a[abs_A - 1])))

In [16]:
# (2): Precedence relations
[mdl.add(mdl.end_before_start(a[i], a[j])) for i, j in E]

# (4): Implication for instantaneous transfers (Delta = 0)
for (i, j, r) in T.keys():
    if Delta[i, j, r] == 0:
        mdl.add(mdl.if_then(f[(i, j, r)] >= 1, mdl.presence_of(z[(i, j, r)])))

# (8): Temporal linking for transfers
for (i, j, r) in z.keys():
    mdl.add(mdl.end_before_start(a[i], z[(i, j, r)]))
    mdl.add(mdl.end_before_start(z[(i, j, r)], a[j]))

# Helper: Store pulse expressions for cumulative constraint (Eq 9)
cumulative_contributions = {(i, j, r): mdl.pulse(z[(i, j, r)], (0, T[(i, j, r)]))
                            for (i, j, r) in T.keys() if Delta[i, j, r] > 0}

# (3), (5), (6), (7): Flow conservation constraints
for i in range(abs_A):
    for r in range(abs_R):
        if Q[i, r] == 0 and i != 0:
            continue
        
        # Build incoming flow expressions
        incoming = []
        for j in range(abs_A):
            if (j, i, r) in z:
                if Delta[j, i, r] == 0:
                    # Instantaneous transfer
                    incoming.append(f[(j, i, r)])
                else:
                    # (5): Flow-height linkage for durative transfers
                    height = mdl.height_at_start(z[(j, i, r)], cumulative_contributions[(j, i, r)])
                    mdl.add(f[(j, i, r)] == height)
                    incoming.append(height)
        
        # Build outgoing flow expressions
        outgoing = []
        for j in range(abs_A):
            if (i, j, r) in z:
                if Delta[i, j, r] == 0:
                    # Instantaneous transfer
                    outgoing.append(f[(i, j, r)])
                else:
                    # (5): Flow-height linkage for durative transfers
                    height = mdl.height_at_start(z[(i, j, r)], cumulative_contributions[(i, j, r)])
                    mdl.add(f[(i, j, r)] == height)
                    outgoing.append(height)
        
        # (3): Source flow initialization
        if i == 0:
            if outgoing:
                mdl.add(mdl.sum(outgoing) == C[r])
        # (6): Flow conservation (into sink activity)
        elif i == abs_A - 1:
            if incoming:
                mdl.add(mdl.sum(incoming) == Q[i, r])
        else:
            # (6): Flow conservation (into activity)
            if incoming:
                mdl.add(mdl.sum(incoming) == Q[i, r])
            # (7): Flow conservation (out of activity)
            if outgoing:
                mdl.add(mdl.sum(outgoing) == Q[i, r])

# (9): Resource capacity (cumulative constraint)
for r in range(abs_R):
    activity_pulses = [mdl.pulse(a[i], Q[i, r]) for i in range(abs_A) if Q[i, r] > 0]
    transfer_pulses = [cumulative_contributions[(i, j, res)] 
                      for (i, j, res) in cumulative_contributions.keys() if res == r]
    if activity_pulses or transfer_pulses:
        mdl.add(mdl.sum(activity_pulses + transfer_pulses) <= C[r])

#### Solve the model

In [17]:
print("Solving model...")
solution = mdl.solve(TimeLimit=300, LogVerbosity='Quiet')
makespan = solution.get_objective_values()[0]
print("\n" + "="*80)
print(f"{'RCPSPTT SOLUTION':^80}")
print("="*80)
print(f"Instance: {filepath.name}")
print(f"Jobs: {abs_A} | Resources: {abs_R} | Makespan: {makespan}")
print("="*80)

Solving model...

                                RCPSPTT SOLUTION                                
Instance: j301_a.sm
Jobs: 32 | Resources: 4 | Makespan: 57


#### Visualisation

In [18]:
# Activities section
print(f"\n{'ACTIVITY SCHEDULE':^80}")
print("-"*80)
print(f"{'Job':<8} {'Start':<10} {'End':<10} {'Duration':<10} {'Resource Demands'}")
print("-"*80)
for i in range(abs_A):
    s, e, dur = solution[a[i]].start, solution[a[i]].end, p[i]
    demands = " | ".join([f"R{r}:{Q[i,r]}" for r in range(abs_R) if Q[i,r] > 0]) or "—"
    print(f"a_{i:<5} {s:<10} {e:<10} {dur:<10} {demands}")
print("-"*80)

# Transfers section
print(f"\n{'RESOURCE TRANSFERS':^80}")
print("-"*80)

active_transfers = []
for (i, j, r) in z.keys():
    try:
        trans, flow = solution[z[(i, j, r)]], solution[f[(i, j, r)]]
        if trans.is_present() and flow > 0:
            active_transfers.append((r, trans.start, i, j, trans.end, int(Delta[i, j, r]), flow))
    except:
        pass

if active_transfers:
    for r in range(abs_R):
        r_transfers = [t for t in active_transfers if t[0] == r]
        if r_transfers:
            print(f"\nResource R{r} (Capacity: {C[r]}):")
            print(f"  {'Transfer':<15} {'Start':<10} {'End':<10} {'Duration':<10} {'Flow'}")
            print(f"  {'-'*60}")
            for _, start, i, j, end, dur, flow in sorted(r_transfers, key=lambda x: x[1]):
                print(f"  z_{i}_{j}_{r:<9} {start:<10} {end:<10} {dur:<10} {flow}")
    print(f"\n  Total transfers: {len(active_transfers)}")
else:
    print("  No active transfers (all instantaneous)")
print("-"*80)

# Summary
print(f"\n{'SOLUTION SUMMARY':^80}")
print("-"*80)
print(f"  • Makespan: {makespan} | Activities: {abs_A} | Active transfers: {len(active_transfers)}")
print(f"  • Resources: {abs_R} | Precedence arcs: {len(E)}")
print("="*80 + "\n")


                               ACTIVITY SCHEDULE                                
--------------------------------------------------------------------------------
Job      Start      End        Duration   Resource Demands
--------------------------------------------------------------------------------
a_0     0          0          0          —
a_1     0          4          4          R0:1
a_2     0          3          3          R3:2
a_3     0          10         10         R3:4
a_4     4          14         10         R2:1
a_5     34         39         5          R1:10
a_6     4          5          1          R1:3
a_7     7          14         7          R0:3
a_8     14         18         4          R3:1
a_9     42         47         5          R3:6
a_10    14         21         7          R3:6
a_11    21         31         10         R1:3
a_12    18         20         2          R1:7
a_13    18         22         4          R2:2
a_14    23         28         5          R1:5
a_15    1

## Aditional Resources

- **Instances for RCPSPTT**
  - https://www2.informatik.uni-osnabrueck.de/kombopt/data/rcpsp/

- **For image of the problem instance see this article:** [An efficient genetic algorithm to solve the resource-constrained project scheduling problem with transfer times](https://www.sciencedirect.com/science/article/pii/S0377221717306549)

- [Problem defintion](https://drive.google.com/file/d/1Gwfgm7mcY47d0zJWLY-SD9BVqHYsUKkP/view?usp=sharing)